Suppose there was data on a website that does not offer the option to save it to a file. One solution is to physically copy and paste all the data into a file.
This is unsustainable for large datasets.

However, we do know that a website is really just HTML that is interepreted by a web browser. Perhaps we can make Python somehow read the elements of tags that contain the data that we want? In addition, we know that HTTP, using a GET request, is able to return the HTML of a website.

With this in mind, we now have an idea: 1. Have Python get the HTML. 2. Make Python understand what in the world it just got. 3. Tell it to grab stuff from the HTML now that it knows what it is.

In [3]:
#This library will help us get the HTML from the website.
import requests

#This library will help Python understand the HTML.
from bs4 import BeautifulSoup

First we get the HTML. Here, "response" is an object representing an HTTP GET request. "text" is an variable of that object that contains the HTML.

In [ ]:
#Get the HTML
response = requests.get('https://www.countrycode.org/')
html = response.text #response.text is the HTML of the website
print(html) #Compare this output to the source code of the website.
#type(html) #This is a string.

But to Python, this html variable is just a bunch of text. It doesn't mean anything special. This is where we use BeautifulSoup.

In [ ]:
#Make BeautifulSoup teach Python about HTML
soup = BeautifulSoup(html, 'html.parser')
print(soup) #This looks the same as the html variable above, but there is one crucial difference: now the HTML tags actually mean something to Python.

Now that Python can read HTML tags, let's plan out how we can use this to our advantage. This part will not be the same every time since different websites will have different HTML. Using the developer tools, we observe that all the data is inside a tag called "tbody". In particular, each row of the table is represented by "tr" and within each row, an individual cell's data is represented by "td".

Because of this, we now have a plan: Go through each of the rows in the table, and then from each row, return the individual cells. A "for" loop could be useful here. The pseudocode for this is listed below.

In [6]:
#Strategy:

#go through each row of the table
    #then grab a cell's data from the row.

#Let's start translating this into Pythonese
    #for each of the rows (tr) in the table (tbody):
        # get the cell's data (td) and store it somewhere, perhaps a dictionary
            # a dictionary could be helpful here because we can name the keys to keep track of where the values are going
        # then store all the dictionaries somewhere (perhaps appending to a list?)    

Before we move on, however, let's understand how we can "drill down" into the table using Python. After all, if we can make this work for 1 element, we can extend the logic to the entire table.

In [ ]:
#It is recommended to uncomment the code underneath each variable assignment to see what each new variable looks like.

table_body = soup.find('tbody') #This returns the first occurrence of <tbody> in the HTML. Fortunately, this is the one that we need.
#table_body
table_rows = table_body.find_all('tr') #This returns a list of <tr> HTML tags. CAUTION: this is a LIST of TAGS, not the HTML itself.
#table_rows
#table_datas = table_rows.find_all('td') #This is wrong. This is because you're trying to call the find_all method on a list. But find_all is the method of an HTML tag.
table_datas = table_rows[0].find_all('td') #This correctly returns a list of <td> HTML tags.
#table_datas
#table_datas[0] #Let's look at the 1st element of this list. This is an HTML tag. However, we want the element inside the tag.
table_datas[0].string #This turns the element of the 1st <td> tag into a string. Bingo. See what happens when you replace the 0 with other indices.

As seen above, we have successfully returned the HTML element that we wanted. Let's take what we've learned and apply it to our Pythonese pseudocode.

In [ ]:
countries = [] #Remember: we would like to store the dictionaries into one list.
for row in table_rows: #Looping through the table.
    table_datas = row.find_all('td') #Finding the <td> tags
    country = {} #initializing the dictionary for each country row, and resetting it to empty after a row is filled out
    country['name'] = table_datas[0].string #the 'name' key of the dictionary now has the name of the country
    country['country code'] = table_datas[1].string #the 'country' key of the dictionary now has the country code
    countrycodes = table_datas[2].string.split('/')#this element of the table_datas list is actually a string with two parts. we would like to split it...
    country['iso-2'] = countrycodes[0] #...put the first element in one dictionary key
    country['iso-3'] = countrycodes[1] #...then put the second element in another dictionary key
    country['population'] = table_datas[3].string #population key now has the population value
    country['area'] = table_datas[4].string #area key now has the area value
    country['gdp'] = table_datas[5].string #gdp key now has the gdp value
    countries.append(country) #appends the country dictionary to the list

countries #We did it!